<a href="https://colab.research.google.com/github/jortegon/Aprendizaje-Profundo/blob/main/Human_horses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip

import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
local_zip = '/tmp/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/validation-horse-or-human')
zip_ref.close()
# Directory with our training horse pictures
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')

# Directory with our training human pictures
train_human_dir = os.path.join('/tmp/horse-or-human/humans')

# Directory with our training horse pictures
validation_horse_dir = os.path.join('/tmp/validation-horse-or-human/horses')

# Directory with our training human pictures
validation_human_dir = os.path.join('/tmp/validation-horse-or-human/humans')

--2024-10-03 22:49:10--  https://storage.googleapis.com/learning-datasets/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.218.207, 142.251.31.207, 142.251.18.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.218.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M  37.4MB/s    in 4.6s    

2024-10-03 22:49:14 (31.3 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]

--2024-10-03 22:49:15--  https://storage.googleapis.com/learning-datasets/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.218.207, 142.251.31.207, 142.251.18.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.218.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/

In [2]:
import tensorflow as tf
#import os

def load_dataset(data_dir, img_height=180, img_width=180, batch_size=32):
    """
    Carga un conjunto de datos de imágenes desde un directorio.

    Args:
        data_dir (str): Ruta al directorio que contiene las subcarpetas de clases.
        img_height (int): Altura a la que se redimensionarán las imágenes.
        img_width (int): Ancho a la que se redimensionarán las imágenes.
        batch_size (int): Tamaño del lote.

    Returns:
        train_dataset: conjunto de datos de TensorFlow.
    """

    # Cargar el conjunto de datos
    train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        seed=123,  # Para reproducibilidad
        image_size=(img_height, img_width),
        batch_size=batch_size
    )

    return train_dataset

# Cargar datos
data_directory = '/tmp/horse-or-human'
train_ds = load_dataset(data_directory)

data_directory2 = '/tmp/validation-horse-or-human'
val_ds = load_dataset(data_directory2)


Found 1027 files belonging to 2 classes.
Found 256 files belonging to 2 classes.


In [3]:
def augment_dataset(dataset):
    """
    Aplica aumentación de datos a un conjunto de datos de imágenes.

    Args:
        dataset: Conjunto de datos de TensorFlow.

    Returns:
        dataset: Conjunto de datos con aumentación aplicada.
    """

    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.RandomRotation(0.2),
        tf.keras.layers.RandomZoom(0.1),
        tf.keras.layers.RandomTranslation(0.1, 0.1),
    ])

    # Aplica la aumentación a las imágenes en el conjunto de datos
    return dataset.map(lambda x, y: (data_augmentation(x, training=True), y))


# Aplica aumentación al conjunto de entrenamiento
train_ds = augment_dataset(train_ds)

# Puedes opcionalmente hacer el preprocesamiento (normalización)
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Normaliza las imágenes
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

# Ajusta la cantidad de datos por lotes y el shuffle
train_ds = train_ds.shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.batch(32).prefetch(tf.data.AUTOTUNE)


In [4]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(180, 180, 3)),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [5]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [6]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 178, 178, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 89, 89, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 87, 87, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 43, 43, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 41, 41, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 20, 20, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 18, 18, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 9, 9, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 7, 7, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 3, 3, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 576)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │         295,424 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 393,377 (1.50 MB)

 Trainable params: 393,377 (1.50 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - accuracy: 0.5515 - loss: 0.6896 - val_accuracy: 0.7812 - val_loss: 0.5455
Epoch 2/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.7108 - loss: 0.5657 - val_accuracy: 0.6641 - val_loss: 0.8920
Epoch 3/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.7530 - loss: 0.4926 - val_accuracy: 0.7422 - val_loss: 1.0246
Epoch 4/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 84s 2s/step - accuracy: 0.8245 - loss: 0.4159 - val_accuracy: 0.7344 - val_loss: 0.9683
Epoch 5/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.8098 - loss: 0.3939 - val_accuracy: 0.5352 - val_loss: 1.8267
Epoch 6/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.8229 - loss: 0.3995 - val_accuracy: 0.7148 - val_loss: 0.9426
Epoch 7/10


In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()